In [20]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from newsapi import NewsApiClient
from datetime import datetime, timedelta
import numpy as np
from numpy.random import seed
seed(1)
import requests
from tensorflow import random
random.set_seed(2)
import hvplot.pandas
%matplotlib inline

In [21]:
load_dotenv()

True

In [22]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")
#print(alpaca_api_key)

In [23]:
#15Min interval
timeframe = "1D"
start_date = pd.Timestamp("2017-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-12-31", tz="America/New_York").isoformat()
#today = pd.Timestamp("2021-01-02", tz="America/New_York").isoformat()
ticker = ["PYPL","SQ","MELI","FISV"]
fintech_ticker = alpaca.get_barset(
    ticker,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df
fintech_ticker.dropna(inplace=True)
fintech_ticker.head(5)
#apple_ticker.dropna(inplace=True)

FISV                                      MELI  \
                             open    high     low  close     volume    open   
time                                                                          
2017-09-05 00:00:00-04:00  61.290  61.355  60.660  60.93  1071708.0  252.60   
2017-09-06 00:00:00-04:00  61.185  61.330  60.585  61.19  1610582.0  257.67   
2017-09-07 00:00:00-04:00  61.465  61.785  61.085  61.41   864020.0  258.92   
2017-09-08 00:00:00-04:00  61.335  61.730  61.015  61.33   851518.0  258.21   
2017-09-11 00:00:00-04:00  61.700  62.235  61.400  62.15   734684.0  265.08   

                                                              PYPL         \
                             high     low   close    volume   open   high   
time                                                                        
2017-09-05 00:00:00-04:00  257.76  251.51  256.63  453957.0  61.26  61.75   
2017-09-06 00:00:00-04:00  259.35  251.92  258.10  365307.0  61.74  61.99   
2017-09-07 00:00:00-04:00  260.95  256.16  258.68  328688.0  61.33  62.13   
2017-09-08 00:00:00-04:00  262.25  258.21  260.08  255764.0  61.94  62.00   
2017-09-11 00:00:00-04:00  286.66  265.07  285.22  973710.0  61.71  63.26   

                                                       SQ                \
                             low  close     volume   open   high    low   
time                                                                      
2017-09-05 00:00:00-04:00  60.58  61.27  4462105.0  25.75  25.82  25.24   
2017-09-06 00:00:00-04:00  60.90  61.25  4246915.0  25.90  26.17  25.72   
2017-09-07 00:00:00-04:00  61.28  62.00  4141248.0  26.12  26.69  25.86   
2017-09-08 00:00:00-04:00  61.10  61.13  3893906.0  26.50  26.99  26.40   
2017-09-11 00:00:00-04:00  61.62  62.64  5970316.0  26.80  27.66  26.78   

                                             
                           close     volume  
time                                         
2017-09-05 00:00:00-04:00  25.77  4478368.0  
2017-09-06 00:00:00-04:00  26.07  4507494.0  
2017-09-07 00:00:00-04:00  26.31  6699110.0  
2017-09-08 00:00:00-04:00  26.53  5567927.0  
2017-09-11 00:00:00-04:00  27.58  7416518.0

In [24]:
fintech_df = pd.DataFrame()
fintech_df["PayPal"] = fintech_ticker["PYPL"]["close"]
fintech_df["Square"] = fintech_ticker["SQ"]["close"]
#fintech_df["Affirm"] = fintech_ticker["AFRM"]["close"]
fintech_df["Mercadolibre"] = fintech_ticker["MELI"]["close"]
#fintech_df["Johnson&Johnson"] = nasdaq_ticker["JNJ"]["close"]
#apple_df = apple_df.pct_change().dropna()
fintech_df.index = fintech_df.index.date 
fintech_df.dropna(inplace=True)
fintech_df.head()

,PayPal,Square,Mercadolibre
2017-09-05,61.27,25.77,256.63
2017-09-06,61.25,26.07,258.10
2017-09-07,62.00,26.31,258.68
2017-09-08,61.13,26.53,260.08
2017-09-11,62.64,27.58,285.22


In [25]:
csv_path = Path("./stock_data/vixe.csv/")
data = pd.read_csv(csv_path,index_col="Date", parse_dates=True, infer_datetime_format=True)
data.sort_index(ascending = True, inplace = True)
data.dropna(inplace=True)
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-09-05,11.75,14.06,11.41,12.23,12.23,0.0
2017-09-06,12.27,12.59,11.35,11.63,11.63,0.0
2017-09-07,11.93,12.07,11.32,11.55,11.55,0.0
2017-09-08,11.87,12.60,11.84,12.12,12.12,0.0
2017-09-11,11.38,11.39,10.51,10.73,10.73,0.0


In [26]:
vix_df = pd.DataFrame()
vix_df["Vix"] = data["Close"]
#vix_df.index = fintech_df.index.date 
vix_df.dropna(inplace=True)
vix_df.tail()

,Vix
Date,
2021-08-17,17.910000
2021-08-18,21.570000
2021-08-19,21.670000
2021-08-20,18.559999
2021-08-23,17.150000


In [27]:
data = fintech_df.join(vix_df, how = "inner")
data.sort_index(inplace=True)
data.head()

,PayPal,Square,Mercadolibre,Vix
2017-09-05,61.27,25.77,256.63,12.23
2017-09-06,61.25,26.07,258.10,11.63
2017-09-07,62.00,26.31,258.68,11.55
2017-09-08,61.13,26.53,260.08,12.12
2017-09-11,62.64,27.58,285.22,10.73


In [28]:
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window):
        features = df.iloc[i : (i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [29]:
window_size = 100
feature_column = 1
target_column = 1
X, y = window_data(fintech_df, window_size, feature_column, target_column)
print (f"X sample values:\n{X[:3]} \n")
print (f"y sample values:\n{y[:3]}")

X sample values:
[[25.77   26.07   26.31   26.53   27.58   27.25   27.48   28.64   28.5
  28.675  28.56   28.48   28.41   28.74   27.46   27.61   28.06   28.48
  28.81   29.38   29.94   30.14   30.76   30.78   31.07   31.32   31.9965
  32.82   32.81   33.22   32.69   32.52   32.1103 32.495  32.52   32.71
  32.9    34.11   35.2    35.75   37.19   36.47   36.08   36.89   37.19
  36.77   36.71   37.97   39.2    39.99   39.75   40.66   41.89   44.18
  45.22   48.07   48.81   48.86   41.0001 42.55   39.12   39.2099 38.22
  36.86   37.595  37.6148 38.7299 38.082  38.31   38.08   36.215  37.02
  37.02   37.76   36.91   35.66   35.87   35.13   35.05   35.74   35.11
  34.67   36.1786 37.31   38.1    41.14   40.77   40.8764 40.84   42.29
  41.25   39.32   40.07   40.26   42.54   45.3    45.02   44.325  45.12
  45.97  ]
 [26.07   26.31   26.53   27.58   27.25   27.48   28.64   28.5    28.675
  28.56   28.48   28.41   28.74   27.46   27.61   28.06   28.48   28.81
  29.38   29.94   30.14   30.76   

In [30]:
print(X)

[[ 25.77    26.07    26.31   ...  44.325   45.12    45.97  ]
 [ 26.07    26.31    26.53   ...  45.12    45.97    45.43  ]
 [ 26.31    26.53    27.58   ...  45.97    45.43    45.51  ]
 ...
 [212.62   227.11   229.51   ... 260.89   259.66   257.41  ]
 [227.11   229.51   229.8601 ... 259.66   257.41   263.05  ]
 [229.51   229.8601 236.58   ... 257.41   263.05   270.44  ]]


In [31]:
split = int(0.6 * len(X))
X_train = X[: split]
X_test = X[split :]
y_train = y[: split]
y_test = y[split :]

In [32]:
from sklearn.preprocessing import MinMaxScaler
x_train_scaler = MinMaxScaler()
y_train_scaler = MinMaxScaler()
x_train_scaler.fit(X_train)
y_train_scaler.fit(y_train)
X_train = x_train_scaler.transform(X_train)
y_train = y_train_scaler.transform(y_train)
X_test = x_train_scaler.transform(X_test)
y_test = y_train_scaler.transform(y_test)
#scaler = MinMaxScaler()
#scaler.fit(X)
#X_train = scaler.transform(X_train)
#X_test = scaler.transform(X_test)
#scaler.fit(y)
#y_train = scaler.transform(y_train)
#y_test = scaler.transform(y_test)

In [33]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
#print (f"X_train sample values:\n{X_train[:5]} \n")
#print (f"X_test sample values:\n{X_test[:5]}")

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [35]:
model = Sequential()

#Adding number_units to the architecture enables the model to far more accurate that with a single digit number
number_units = 100
dropout_fraction = 0.2


# Layer 1
model.add(LSTM(units=number_units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(dropout_fraction))
# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))
# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))
# Output layer
model.add(Dense(1))

In [36]:
model.compile(optimizer="adam", loss="mean_squared_error")

In [37]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100, 100)          40800     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 100)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100, 100)          80400     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100, 100)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

In [38]:
model.fit(X_train, y_train, epochs=20, shuffle=False, batch_size=100, verbose=1)

Epoch 1/20
6/6 [==============================] - 6s 316ms/step - loss: 0.0988
Epoch 2/20
6/6 [==============================] - 2s 317ms/step - loss: 0.0260
Epoch 3/20
6/6 [==============================] - 2s 325ms/step - loss: 0.0187
Epoch 4/20
6/6 [==============================] - 2s 331ms/step - loss: 0.0137
Epoch 5/20
6/6 [==============================] - 2s 321ms/step - loss: 0.0159
Epoch 6/20
6/6 [==============================] - 2s 319ms/step - loss: 0.0135
Epoch 7/20
6/6 [==============================] - 2s 343ms/step - loss: 0.0118
Epoch 8/20
6/6 [==============================] - 2s 387ms/step - loss: 0.0117
Epoch 9/20
6/6 [==============================] - 2s 377ms/step - loss: 0.0112
Epoch 10/20
6/6 [==============================] - 2s 346ms/step - loss: 0.0114
Epoch 11/20
6/6 [==============================] - 2s 339ms/step - loss: 0.0102
Epoch 12/20
6/6 [==============================] - 3s 458ms/step - loss: 0.0097
Epoch 13/20
6/6 [==============================] 

In [39]:
model.evaluate(X_test, y_test)

12/12 [==============================] - 2s 60ms/step - loss: 0.3251


0.3250727653503418

In [40]:
predicted = model.predict(X_test)

In [41]:
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

NameError: name 'scaler' is not defined

In [621]:
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
    }, index = fintech_df.index[-len(real_prices): ])
stocks.tail(10)

,Real,Predicted
2021-08-10,938.195309,768.752930
2021-08-11,929.262516,773.904785
2021-08-12,926.956098,776.909302
2021-08-13,910.628124,778.000977
2021-08-16,901.146183,776.918396
2021-08-17,896.643177,773.775757
2021-08-18,888.405970,769.018188
2021-08-19,909.053902,762.912109
2021-08-20,936.108550,756.931274
2021-08-23,935.120085,752.642029


In [622]:
stocks.hvplot.line(label = "PayPal LSTM Model", width = 1000, xlabel="Date", ylabel="Price")
#(label = "Predicting Apple's Close with the S&P500 Close - (6Day Window)", xlabel="Date", ylabel="Price")

:NdOverlay   [Variable]
   :Curve   [index]   (value)